In [5]:
import torch
import torchvision
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

In [6]:
train_data=torchvision.datasets.CIFAR10(root='./datasets',train=True,download=True,transform=transforms.ToTensor())
test_data=torchvision.datasets.CIFAR10(root='./datasets',train=False,download=True,transform=transforms.ToTensor())

In [7]:
len(train_data),len(test_data)

(50000, 10000)

In [8]:
device=torch.device('cpu' if not torch.cuda.is_available() else 'cuda')

In [9]:
class myModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model=nn.Sequential(
        nn.Conv2d(3,32,5,padding=2),
        nn.MaxPool2d(2),
        nn.Conv2d(32,32,5,padding=2),
        nn.MaxPool2d(2),
        nn.Conv2d(32,64,5,padding=2),
        nn.MaxPool2d(2),
        nn.Flatten(),
        nn.Linear(1024,64),
        nn.Linear(64,10))
    def forward(self,x):
        return self.model(x)

In [10]:
train_data_loader=DataLoader(train_data,batch_size=64,shuffle=True)
test_data_loader=DataLoader(test_data,batch_size=64,shuffle=True)

In [11]:
model=myModel()
model.to(device)
loss_fn=nn.CrossEntropyLoss()
loss_fn.to(device)
learning_rate=1e-2
epoch=10
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [12]:
total_train_times=0
total_test_times=0

In [13]:
writer=SummaryWriter('log8')

In [14]:
for i in range(1,epoch+1):
    print(f'-------------epoch:{i} begining---------------')
    for data in train_data_loader:
        imgs,labels=data
        imgs=imgs.to(device)
        labels=labels.to(device)
        outputs=model(imgs)
        loss=loss_fn(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_times+=1
        if total_train_times%100==0:
            print('total_train_times:',total_train_times,'total_loss:',loss)
    total_loss=0
    with torch.no_grad():
        for data in test_data_loader:
            imgs,labels=data
            imgs=imgs.to(device)
            labels=labels.to(device)
            outputs=model(imgs)
            loss=loss_fn(outputs,labels)
            total_loss+=loss
    total_test_times+=1
    print('total_loss:',total_loss)
    writer.add_scalar('test_loss',total_loss,total_train_times)

-------------epoch:1 begining---------------
total_train_times: 100 total_loss: tensor(2.2922, device='cuda:0', grad_fn=<NllLossBackward0>)
total_train_times: 200 total_loss: tensor(2.2721, device='cuda:0', grad_fn=<NllLossBackward0>)
total_train_times: 300 total_loss: tensor(2.1877, device='cuda:0', grad_fn=<NllLossBackward0>)
total_train_times: 400 total_loss: tensor(2.1485, device='cuda:0', grad_fn=<NllLossBackward0>)
total_train_times: 500 total_loss: tensor(2.1164, device='cuda:0', grad_fn=<NllLossBackward0>)
total_train_times: 600 total_loss: tensor(1.9521, device='cuda:0', grad_fn=<NllLossBackward0>)
total_train_times: 700 total_loss: tensor(2.2426, device='cuda:0', grad_fn=<NllLossBackward0>)
total_loss: tensor(303.5390, device='cuda:0')
-------------epoch:2 begining---------------
total_train_times: 800 total_loss: tensor(1.8278, device='cuda:0', grad_fn=<NllLossBackward0>)
total_train_times: 900 total_loss: tensor(1.9261, device='cuda:0', grad_fn=<NllLossBackward0>)
total_tra

KeyboardInterrupt: 